#### Llama2 기본 모델을 로딩하여 문장 생성
* 모델이름: llama-2-7b
* 설명 예제(GitHub)
    1. https://github.com/facebookresearch/llama-recipes/tree/main/examples
    2. https://github.com/facebookresearch/llama-recipes/blob/main/examples/inference.py

In [7]:
import torch
from transformers import LlamaForCausalLM, LlamaTokenizer


ModuleNotFoundError: No module named 'llama_recipes'

In [1]:
config = {
    "peft_model" : "None",
    "quantization": False,
    "max_new_tokens": 100,  #The maximum numbers of tokens to generate
    "prompt_file": "None",
    "seed": 42,  #seed value for reproducibility
    "do_sample": True,  #Whether or not to use sampling ; use greedy decoding otherwise.
    "min_length": None, #The minimum length of the sequence to be generated, input prompt + min_new_tokens
    "use_cache": True, #[optional] Whether or not the model should use the past last key/values attentions Whether or not the model should use the past last key/values attentions (if applicable to the model) to speed up decoding.
    "top_p": 1.0, # [optional] If set to float < 1, only the smallest set of most probable tokens with probabilities that add up to top_p or higher are kept for generation.
    "temperature": 1.0, # [optional] The value used to modulate the next token probabilities.
    "top_k": 50,  # [optional] The number of highest probability vocabulary tokens to keep for top-k-filtering.
    "repetition_penalty": 1.0,  #The parameter for repetition penalty. 1.0 means no penalty.
    "length_penalty": 1, #[optional] Exponential penalty to the length that is used with beam-based generation. 
    "enable_azure_content_safety": False, # Enable safety check with Azure content safety api
    "enable_sensitive_topics": False, # Enable check for sensitive topics using AuditNLG APIs
    "enable_salesforce_content_safety": True, # Enable safety check with Salesforce safety flan t5
    "enable_llamaguard_content_safety": False, 
    "max_padding_length": None, # the max padding length to be used with tokenizer padding the prompts.
    "use_fast_kernels": False # Enable using SDPA from PyTroch Accelerated Transformers, make use Flash Attention and Xformer memory-efficient kernels    
}

In [2]:
model_id = "/data/bwllm/models/llama-2-7b"
tokenizer = LlamaTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

NameError: name 'LlamaTokenizer' is not defined

In [3]:
model = load_model(model_id, config['quantization'], config['use_fast_kernels'])

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(30003, 5120)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-39): 40 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=5120, out_features=15360, bias=True)
          (dense): Linear(in_features=5120, out_features=5120, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=5120, out_features=20480, bias=True)
          (dense_4h_to_h): Linear(in_features=20480, out_features=5120, bias=True)

In [4]:
print(tokenizer)

PreTrainedTokenizerFast(name_or_path='/data/bwllm/models/KoAlpaca-Polyglot-12.8B', vocab_size=30000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<|endoftext|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|endoftext|>', '<|sep|>', '<|acc|>', '<|tel|>', '<|rrn|>']}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<|unused0|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<|unused1|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<|sep|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	30000: AddedToken("<|acc|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	30001: Add

In [5]:
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))
    tokenizer.pad_token = tokenizer.eos_token
    
tokenizer.pad_token = tokenizer.eos_token    

In [6]:
query = "대한민국 수도는 어디지"
inputs = tokenizer.encode_plus(query, return_tensors="pt")
input_ids = inputs["input_ids"]
attention_mask = inputs["attention_mask"]
input_ids

tensor([[7635, 2248,  272, 2116,  295]])

In [7]:
generate_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_new_tokens=512)
generate_ids

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


In [23]:
generated_text = tokenizer.decode(generate_ids[0], skip_special_tokens=True)
generated_text

TypeError: argument 'ids': 'list' object cannot be interpreted as an integer